In [32]:
import pandas as pd
import plotly
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

# Remote data access 

- JHU
- ECDC 

In [33]:
#country_concord= pd.read_excel("ecdc country UN region concorded 20200416.xlsx")
#country_concord.to_csv("ecdc country UN region concorded 20200416.csv", encoding='utf8', index=False)
country_concord_test= pd.read_csv("ecdc country UN region concorded 20200416.csv", encoding='utf8')
ecdc= pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv", na_values= "", encoding = "UTF-8")

In [34]:
country_concord_test[ country_concord_test.iso3=="CIV"]

,countriesAndTerritories,country,iso3,unregion,unsubregion
43,Cote_dIvoire,Côte d'Ivoire,CIV,Africa,Western Africa


In [20]:
ecdc= ecdc[ecdc.year==2020]
ecdc.drop(columns=["month", "year", "day", "geoId", "countryterritoryCode"], inplace= True)
ecdc.dateRep= pd.to_datetime(ecdc.dateRep, format='%d/%m/%Y')
ecdc["date"]= ecdc["dateRep"].astype('str')
ecdc= pd.merge(left= ecdc, right= country_concord, on=["countriesAndTerritories"], how="left")
# jhu= pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

In [21]:
ecdc.head(3)

,dateRep,cases,deaths,countriesAndTerritories,popData2018,date,country,iso3,unregion,unsubregion
0,2020-04-16,70,2,Afghanistan,37172386.0,2020-04-16,Afghanistan,AFG,Asia,Southern Asia
1,2020-04-15,49,2,Afghanistan,37172386.0,2020-04-15,Afghanistan,AFG,Asia,Southern Asia
2,2020-04-14,58,3,Afghanistan,37172386.0,2020-04-14,Afghanistan,AFG,Asia,Southern Asia


In [4]:
jhu.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7,7,11,14,14,15,15,18,18,21
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,20,20,21,22,22,23,23,23,23,23
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,130,152,173,193,205,235,256,275,293,313


# Country Region Concordance

In [49]:
'''
mis= pd.read_excel("Diplometrics_Master_MIS.xlsx")
un_region= pd.read_excel("Diplometrics_Master_MIS.xlsx", sheet_name='UN Region')
ecdc_c= ecdc[['countriesAndTerritories', 'geoId', 'countryterritoryCode']].drop_duplicates()
'''
'''
mis_cname= set(mis["Country"].dropna())
un_region_cname= set(un_region.Country.dropna())
for c in mis_cname:
    if c not in un_region_cname:
        print(c)
'''
'''
un_region= pd.merge(left=un_region, right=mis[['Country', 'genc_3']], how="left", on='Country')
del mis, mis_cname
'''
'''
un_ccode= set(un_region.genc_3.dropna())
ecdc_ccode= set(ecdc_c.countryterritoryCode.dropna())
for i in ecdc_ccode:
    if i not in mis_ccode:
        print(i)
'''
#ecdc_c[(ecdc_c.geoId.isna()) | (ecdc_c.countryterritoryCode.isna())]
#un_region_code= pd.merge(left=un_region.dropna(), right= ecdc_c.dropna(), left_on='genc_3', right_on='countryterritoryCode', how='right')
#un_region_code.to_excel("ecdc country UN region initial.xlsx", index=False)

# Different Measures

- Total Deaths
- Deaths/Pop
- Mortality Rate: Deaths per 1,000 people individuals per year

In [22]:
df= ecdc.copy(True)

In [23]:
fn = lambda row: 100*row.deaths/row.popData2018 # define a function for the new column
# mn = lambda row: 365*1000*row.deaths/row.popData2018
col = df.apply(fn, axis=1) # get column data with an index
df = df.assign(death_per_capita=col.values) # assign values to new column
df.sort_values(by=["countriesAndTerritories","dateRep"], inplace=True)
df.reset_index(drop=True, inplace=True)
df["total_deaths"]=df.groupby(["countriesAndTerritories"])["deaths"].cumsum()
df["death_per_capita_7dayMA"]=df.groupby(["countriesAndTerritories"])["death_per_capita"].rolling(window=7,min_periods=1).mean().reset_index(drop=True)
df[df.countriesAndTerritories=="China"].tail(5)

,dateRep,cases,deaths,countriesAndTerritories,popData2018,date,country,iso3,unregion,unsubregion,death_per_capita,total_deaths,death_per_capita_7dayMA
2262,2020-04-12,93,0,China,1.392730e+09,2020-04-12,China,CHN,Asia,Eastern Asia,0.000000e+00,3343,1.025735e-07
2263,2020-04-13,112,2,China,1.392730e+09,2020-04-13,China,CHN,Asia,Eastern Asia,1.436029e-07,3345,1.025735e-07
2264,2020-04-14,94,0,China,1.392730e+09,2020-04-14,China,CHN,Asia,Eastern Asia,0.000000e+00,3345,1.025735e-07
2265,2020-04-15,49,1,China,1.392730e+09,2020-04-15,China,CHN,Asia,Eastern Asia,7.180143e-08,3346,9.231612e-08
2266,2020-04-16,50,0,China,1.392730e+09,2020-04-16,China,CHN,Asia,Eastern Asia,0.000000e+00,3346,7.180143e-08


# Change date to the day when first 3 deaths recorded

In [7]:
county_list= list(df.countriesAndTerritories.drop_duplicates())
dt_start_date_list= []
for country in county_list:
    dt= df[df.countriesAndTerritories==country].copy(True)
    if dt[dt.deaths>=3].empty:
        continue
    first_3dailydeath= dt[dt.deaths>=3].index.values[0]
    dt= dt.loc[first_3dailydeath:,:]
    dt["number_of_days"]= range(1,1+dt.shape[0])
    dt_start_date_list.append(dt)
df_numofday_3dailydead= pd.concat(dt_start_date_list)
df_numofday_3dailydead.head(5)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018,death_per_capita,total_deaths,death_per_capita_7dayMA,number_of_days
88,2020-04-08,8,4,2020,30,4,Afghanistan,AF,AFG,37172386.0,0.000011,11,0.000003,1
89,2020-04-09,9,4,2020,56,3,Afghanistan,AF,AFG,37172386.0,0.000008,14,0.000004,2
90,2020-04-10,10,4,2020,61,1,Afghanistan,AF,AFG,37172386.0,0.000003,15,0.000004,3
91,2020-04-11,11,4,2020,37,0,Afghanistan,AF,AFG,37172386.0,0.000000,15,0.000004,4
92,2020-04-12,12,4,2020,34,3,Afghanistan,AF,AFG,37172386.0,0.000008,18,0.000005,5


# Specific Countries & Graph

In [8]:
dt_graph= df_numofday_3dailydead.copy(True)
dt_graph.rename(columns={"countriesAndTerritories": "country_or_territory", "popData2018": "population_2018",
                        "deaths": "daily_deaths", "cases": "daily_cases"}, inplace=True)
dt_graph["date"]=dt_graph['dateRep'].copy().astype(str)

In [93]:
# dt_graph[[ "countriesAndTerritories", "total_deaths" ]].drop_duplicates(subset=["countriesAndTerritories"], keep= "last").sort_values("total_deaths").to_excel("covid total deaths 20200414.xlsx")

In [9]:
countries_default_group= ['United_States_of_America', 'China', 'South_Korea', 'Spain', 'Italy', 
                          'Nigeria', 'Egypt', 'Algeria', 'Ethiopia',
                           'Morocco', 'Burkina_Faso', 'South_Africa']
country_region= {'United_States_of_America': "North America", 
                 'China': "Asia", 'South_Korea': 'Asia', 
                 'Spain': 'Europe', 'Italy': 'Europe', 
                'Nigeria': 'Africa', 'Egypt': 'Africa', 'Algeria': 'Africa', 'Ethiopia': 'Africa',
                'Morocco': 'Africa', 'Burkina_Faso': 'Africa', 'South_Africa': 'Africa'}
dt_sub= dt_graph[dt_graph['country_or_territory'].isin(countries_default_group)].copy(True)
dt_sub["region"]= dt_sub['country_or_territory'].replace(country_region)

In [67]:
# Hidden high values traces 
fig = go.Figure()
#
for c in countries_default_group:
    dt_country= dt_sub[dt_sub.country_or_territory==c]
    if not dt_country.empty:
        dt_country['h_text']= dt_country.apply(lambda x: f'{x.country_or_territory};\n{x.date};\nDaily deaths: {x.daily_deaths};\nTotal deaths: {x.total_deaths}', axis=1).copy(True)
        #
        if dt_country.region.values[0] == "Africa":
            fig.add_trace(go.Scatter(
                x=dt_country['number_of_days'], y=dt_country['death_per_capita_7dayMA'],
                mode='lines', name= c, 
                hovertext= dt_country['h_text']
                                    ))
        else:
            fig.add_trace(go.Scatter(
                x=dt_country['number_of_days'], y=dt_country['death_per_capita_7dayMA'],
                mode='lines', name= c, visible='legendonly',
                hovertext= dt_country['h_text']
                                    ))
#
fig.update_layout(
    template= 'plotly_white',
    title= 'COVID-19: Daily deaths per Population- 7 day moving avg',
    xaxis_title= "Number of Days- since 3 daily deaths recorded",
    showlegend=True
                )
#
plotly.offline.plot(fig, filename = 'COVID-19 Deaths per capita 7-day Moving Avg in Selected Countries.html', auto_open=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



'COVID-19 Deaths per capita 7-day Moving Avg in Selected Countries.html'

# Other Graph

In [84]:
dt_example = px.data.gapminder()
dt_example.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [6]:
df_sub.columns = ['Date','day','month','year','cases','daily_deaths','Country','geold','countryterritoryCode',
                     '2018Population','death_per_capita',"total_deaths","death_per_capita_7dayMA"]  #change column names
df_sub["Date_str"]=df_sub['Date'].copy().astype(str)

In [7]:
fig = px.line(df_sub, x='Date_str', y='death_per_capita_7dayMA'
                 , hover_data=['Date_str', 'Country', 'daily_deaths', 'total_deaths', 'death_per_capita_7dayMA', '2018Population']
                 , color= 'Country'
                 ,width=1200, height=800)
fig.update_layout(template='plotly_white')
fig.update_layout(title='COVID-19 Deaths per capita 7-day Moving Avg in Selected Countries')
plotly.offline.plot(fig, filename = 'COVID-19 Deaths per capita 7-day Moving Avg in Selected Countries.html', auto_open=False)

'COVID-19 Deaths per capita 7-day Moving Avg in Selected Countries.html'